<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/0_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [ ]:
!pip install -q pydrive2

# Downloading the Files

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive2 client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from os import makedirs
from os.path import dirname, join

def download_drive_item(drive, item_id, dest_path):
    """
    Downloads a file or folder from Google Drive to the specified local destination.

    Args:
      drive (GoogleDrive): Authenticated GoogleDrive instance.
      item_id (str): The ID of the Google Drive item (file or folder) to download.
      dest_path (str): The local destination path to download the item.

    """
    item = drive.CreateFile({'id': item_id})
    item.FetchMetadata()

    # Determine if the item is a folder or a file
    if item['mimeType'] == 'application/vnd.google-apps.folder':
        makedirs(dest_path, exist_ok=True)
        file_list = drive.ListFile({'q': f"'{item_id}' in parents and trashed=false"}).GetList()
        for file in file_list:
            file_path = join(dest_path, file['title'])
            if file['mimeType'] == 'application/vnd.google-apps.folder':
                download_drive_item(drive, file['id'], file_path)
            else:
                makedirs(dirname(file_path), exist_ok=True)
                print(f"Downloading {file['title']} to {file_path}")
                file.GetContentFile(file_path)
    else:
        makedirs(dirname(dest_path), exist_ok=True)
        print(f"Downloading {item['title']} to {dest_path}")
        item.GetContentFile(dest_path)

#### IFSP Boituva Portarias

In [ ]:
item_id = '1NX5kRKEGV3t5X0k_wb3RJHNbCLASiiiw'
destination_path = '/content/shared_item'
download_drive_item(drive, item_id, destination_path)

### Estatuto

In [ ]:
file_id = '1XzjBDrh0N3TxUbKc4Jjs5pEaONxj7lx2'
downloaded_file = drive.CreateFile({'id': file_id})
downloaded_file.GetContentFile('ESTATUTO IFSP-VERSÃO VIGENTE-Resolução 08-2014--ALERAÇÃO da Resolução 872-2013.pdf')